### To install mumps tap on terminal:
```shell
sudo apt install libmumps-ptscotch-dev && pip install pymumps
```

In [103]:
#%%file poisson_2d.py
from manapy.ddm import readmesh
from manapy.ddm import Domain

from manapy.ast import Variable, LinearSystem
import numpy as np

from  numpy.linalg import solve

from scipy import sparse
from scipy.sparse.linalg import spsolve

import os

from triangle import dist, orthocenter

from matplotlib import pyplot, cm
from mpl_toolkits.mplot3d import Axes3D

from mpi4py import MPI
COMM = MPI.COMM_WORLD
SIZE = COMM.Get_size()
RANK = COMM.Get_rank()

# ... get the mesh directory
'''try:
    MESH_DIR = os.environ['MESH_DIR']
 
except:
    BASE_DIR = os.path.dirname(os.path.realpath(__file__))
    BASE_DIR = os.path.join(BASE_DIR , '..')
    MESH_DIR = os.path.join(BASE_DIR, 'mesh')
'''
filename = "carre.msh"
#filename = os.path.join(MESH_DIR, filename)

        
dim = 2
readmesh(filename, dim=dim, periodic=[0,0,0])
    
#Create the informations about cells, faces and nodes
domain = Domain(dim=dim)
    
cells   = domain.cells
nbcells = domain.nbcells
faces   = domain.faces
nodes   = domain.nodes
nbfaces = len(faces.mesure)

def Assembly_M1(cells, nbcells, faces, nodes):
    
    row    = []
    col    = []
    values = []
    b = np.zeros(nbcells, dtype = np.float64 ) 
    for i in range(nbcells):
        # Compute orthocenter of current cell i
        x_k = orthocenter([ nodes.vertex[ cells.nodeid[i][k] ] for k in range(3) ])
        
        s = 0            
        for e in cells.faceid[i]:
            if faces.name[e] == 0:
                for j in cells.cellfid[i]:
                    if e in cells.faceid[j]:
                        x_L = orthocenter([ nodes.vertex[ cells.nodeid[j][k] ] for k in range(3) ])
                        p = faces.mesure[e] / dist(x_k, x_L)
                        row.append(i)
                        col.append(j)
                        values.append(p)
                        s -= p
            if faces.name[e] == 1:
                p = faces.mesure[e] / dist(x_k, faces.center[e])
                s -= p
                b[i] = - (20 * p)
            if faces.name[e] in  [2, 3, 4]:
                p = faces.mesure[e] / dist(x_k, faces.center[e])
                s -= p
        row.append(i)
        col.append(i)
        values.append(s)    
        
    mat = sparse.csr_matrix((values, (row, col)))
    
    return mat, b
                
def Assembly_M2(cells, nbcells, faces, nodes):
    
    center = np.zeros( (nbcells, 2), dtype = np.float64 )
    b      = np.zeros(  nbcells, dtype = np.float64 ) 
    A      = np.zeros( (nbcells, nbcells), dtype = np.float64 )
    
    # Compute orthocenter for all cells
    for i in range(nbcells):
        center[i, :] = orthocenter([ nodes.vertex[ cells.nodeid[i][k] ] for k in range(3) ])
    
    for i in range(nbfaces):
        l_cell = faces.cellid[i][0]
        r_cell = faces.cellid[i][1]
        
        # Innerfaces
        if faces.name[i] == 0:
            p = faces.mesure[i] / dist(center[l_cell], center[r_cell])
            A[l_cell, l_cell] -= p
            A[r_cell, r_cell] -= p
            A[l_cell, r_cell]  = p
            A[r_cell, l_cell]  = p
        
        # in faces    
        if faces.name[i] == 1:
            for j in [l_cell, r_cell]:
                if j != -1:
                    p = faces.mesure[i] / dist(center[j], faces.center[i])
                    A[j, j] -= p
                    b[j]     = - (20 * p)
        
        # out, upper, bottom faces
        if faces.name[i] in [2, 3, 4]:
            for j in [l_cell, r_cell]:
                if j != -1:
                    p = faces.mesure[i] / dist(center[j], faces.center[i])
                    A[j, j] -= p 
            
        
    return A, b

def Assembly_M3(cellid:'int[:,:]', centerf:'float[:]', 
                mesuref:'float[:]', namef:'int[:]', innerfaces:'int[:]', 
                boundaryfaces:'int[:]'):
    
    sizeM  = 4*len(innerfaces)+len(boundaryfaces)
    center = np.zeros( (nbcells, 2), dtype = np.float64 )
    row    = np.zeros(    sizeM, dtype = np.int32   )
    col    = np.zeros(    sizeM, dtype = np.int32   )
    data   = np.zeros(    sizeM, dtype = np.float64   )
    
    for i in range(nbcells):
        center[i, :] = orthocenter([ nodes.vertex[ cells.nodeid[i][k] ] for k in range(3) ])
        
    cmpt = 0
    for face in innerfaces:
        K = cellid[face][0] 
        L = cellid[face][1]
        mesure = mesuref[face]
        
        row[cmpt] = K
        col[cmpt] = K
        data[cmpt] = - (mesure/ dist(center[K], center[L]))
        cmpt = cmpt + 1
        
        row[cmpt] = L
        col[cmpt] = L
        data[cmpt] = -(mesure/dist(center[K], center[L])) 
        cmpt = cmpt + 1
        
        row[cmpt] = K
        col[cmpt] = L
        data[cmpt] = (mesure/dist(center[K], center[L])) 
        cmpt = cmpt + 1
        
        row[cmpt] = L
        col[cmpt] = K
        data[cmpt] = (mesure/dist(center[K], center[L])) 
        cmpt = cmpt + 1
        
    for face in boundaryfaces: 
        K = cellid[face][0] 
        mesure = mesuref[face]
        
        row[cmpt] = K
        col[cmpt] = K
        data[cmpt] = -(mesure/dist(center[K], centerf[face])) 
        cmpt = cmpt + 1
        
        if namef[face] == 1:
            b[K] = -20*(mesure/dist(center[K], centerf[face])) 
            
    mat = sparse.csr_matrix((data, (row, col)))
    
    return mat, row, col, data, b

 
M1,b1 = Assembly_M1(cells, nbcells, faces, nodes)
M2,b2 = Assembly_M2(cells, nbcells, faces, nodes)
M3, row, col, data, b3 = Assembly_M3(faces.cellid, faces.center, faces.mesure, 
                                    faces.name, domain.innerfaces, domain.boundaryfaces)


u1 = spsolve(M1, b1)
u2 = solve(M2, b2)
u3 = spsolve(M3, b3)

#assert (b1 == b3).all()
#for i in range(M2.shape[0]):
#    for j in range(M2.shape[1]):
#        if M3.toarray()[i][j] - M2[i][j] != 0:
#            print(M3.toarray()[i][j], M2[i][j])
#print(M3.toarray())
#for i in range(nbcells):
#    if np.fabs(u1[i] - u3[i]) > 1e-12:
#        print(u1[i]- u3[i])

Starting ....
Number of Cells :  220
Number of Nodes :  129


In [125]:
import mumps

ctx = mumps.DMumpsContext(comm=COMM)
ctx.set_shape(nbcells)
ctx.set_silent()

ctx.set_distributed_assembled_rows_cols(row+1, col+1)
ctx.set_distributed_assembled_values(data)
ctx.set_icntl(18,3)
    
if COMM.Get_rank() == 0:
    ctx.id.n = nbcells
    u4 = b3.copy()
       
#Analyse 
ctx.run(job=1)
#Factorization Phase
ctx.run(job=2)

#Allocation size of rhs
if COMM.Get_rank() == 0:
    ctx.set_rhs(u4)
else :
    u4 = np.zeros(nbcells)
            
#Solution Phase
ctx.run(job=3)

#Destroy
ctx.destroy()

#for i in range(nbcells):
#    print(u3[i], sol[i])

In [117]:
domain.save_on_cell(miter=0, value = u1)
domain.save_on_cell(miter=1, value = u2)
domain.save_on_cell(miter=2, value = u3)
domain.save_on_cell(miter=2, value = u4)

 **************************** Computing ****************************
$$$$$$$$$$$$$$$$$$$$$$$$$$$$ Saving Results $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Iteration =  0 time =  0 time step =  0
max w = 18.55285597310832
 **************************** Computing ****************************
$$$$$$$$$$$$$$$$$$$$$$$$$$$$ Saving Results $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Iteration =  0 time =  0 time step =  0
max w = 18.552855973108315
 **************************** Computing ****************************
$$$$$$$$$$$$$$$$$$$$$$$$$$$$ Saving Results $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Iteration =  0 time =  0 time step =  0
max w = 18.552855973108315
 **************************** Computing ****************************
$$$$$$$$$$$$$$$$$$$$$$$$$$$$ Saving Results $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Iteration =  0 time =  0 time step =  0
max w = 18.552855973108315
